## Task 1

### 8 Queens Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The goal for task 1 is to use genetic algorithms to obtain a fast solution to the 8 queens problem. The first task will start at n = 8, that is, completing the problem for a standard chessboard.

Using the EightQueensState class from below 

In [13]:
import numpy as np
import random

random.seed(0)

class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        if state is None:
            self.n = n
            state = np.random.randint(0, n, n)
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))

    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            # for each queen, look in columns to the right
            # add one to the count if there is another queen in the same row
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()

            # add one to the count for each queen on the upper or lower diagonal
            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
        return count
    
    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)

    def is_goal(self):
        return self.cost() == 0

    def __str__(self):
        if self.is_goal():
            return f"{self.state} Goal state!"
        else:
            return f"{self.state} cost {self.cost()}"

In [43]:
# using fitness function from course material, generalising to board size n
# using the in-built cost function
def fitness(pop, N):
    fit = np.zeros(N)
    for i in range(len(fit)):
        fit[i] = (n*(n-1))/2 - int(pop[i].cost())
        
    tot = sum(fit)
    fit = fit/tot
    return fit

n = 8
N = 18

# N=population size, n=chess board size
def initialise_population(N, n):
    population = []
    for i in range(N):
        population.append(EightQueensState(n=n))
    
    return population

def print_state(pop):     
    for item in pop:
        print(item) 

# selects N/2 pairs (replaces adult population) of individuals for crossover
# weighted selection of individuals based on their fitness function results
def selection(pop, fitnessout):
    pairs = []
    for i in range(N//2):
        pairs.append(random.choices(pop, weights = fitnessout, k=2))
    return pairs

# crossover - happens at random point to maintain diversity    
def crossover(x, y):
    c1 = []
    c2 = []
    split = random.randrange(0,n)
    c1 = list(x.state[0:split]) + list(y.state[split:n])
    c2 = list(y.state[0:split]) + list(x.state[split:n])
    children = [EightQueensState(state = c1), EightQueensState(state = c2)]
    return children

# mutation function, at 10% rate
def mutate(pop):
    for j in range(len(pop)):
        if random.random() >= 0.90:
            pop[j].state[random.randrange(0,n)] = random.randrange(0,n)
        else:
            pass
    
def contains_solution(pop):
    for state in pop:
        if state.cost() == 0:
            return True
        else:
            pass
    
    return False

# start off w population
# loop begins
# calculate fitnesses of population
# if winner -> return it
# else select parent pairs
# create children
# mutate
# input new population into loop
population = initialise_population(N, n)
print("---------- Generation: 0 --------------")
print_state(population)
i = 1

while contains_solution(population) == False:
    if i == 1:
        print("Fitness of the population is measured using n*(n-1)/2 - cost, where n = 8 for a standard chessboard")
        print("A sample of the fitness weighting table can be seen below:")
        fit = fitness(population, N)
        print(fit)
        print("The parents are selected based on the weights from the fitnesses to make lower cost parents more likely")
        print("However, high cost parents are still possible which keeps diversity to help converge to a solution")
        print("The selection forms N/2 pairs to crossover, where N is the population size. In this case, N = " + str(N))
        print("It was found that increasing the population size increases the speed at which a solution is found")
        print("This comes at the cost of increased space complexity")
        print("The selection pairs for the first generation are sampled below")
        select = selection(population, fit)
        for item in select:
            print_state(item)
        newpopulation = []
        for j in select:
            children = crossover(j[0], j[1])
            newpopulation.append(children[0])
            newpopulation.append(children[1])
        print("Crossover is completed at a random point between the two parents")
        print("The children are then considered the new population")
        print_state(newpopulation)
        population = newpopulation
        mutate(population)
        print("Each individual has a 10% chance to mutate, where a random column changes to a random value")
        print("10% was chosen as it stops a population from becoming homogeneous as it includes higher variance\n")
        
        i+=1
    else:
        if i % 500 == 0:
            print(str(i) + " Generations completed")
        fit = fitness(population, N)
        select = selection(population, fit)
        newpopulation = []
        for j in select:
            children = crossover(j[0], j[1])
            newpopulation.append(children[0])
            newpopulation.append(children[1])

        population = newpopulation
        mutate(population)
        #print("---------- Generation: " + str(i) + " --------------")
        #print_state(population)
        i += 1
    
for state in population:
    if state.is_goal():
        print("Goal State reached after " + str(i) + " Generations")
        print(state)


---------- Generation: 0 --------------
[3 3 1 1 2 4 2 6] cost 9
[6 5 2 3 2 6 6 1] cost 13
[0 1 3 4 6 4 1 2] cost 7
[7 5 2 6 0 6 6 6] cost 8
[0 7 0 7 4 2 7 0] cost 9
[0 6 4 7 5 0 1 0] cost 7
[6 1 7 4 4 1 7 7] cost 10
[3 0 7 3 4 2 3 4] cost 9
[3 2 1 5 0 4 6 0] cost 5
[4 5 6 3 1 2 6 5] cost 9
[5 3 2 3 6 1 1 6] cost 7
[1 6 4 7 6 0 6 6] cost 8
[1 4 1 7 5 1 2 6] cost 6
[1 1 1 3 1 1 1 5] cost 17
[2 3 7 7 0 4 7 7] cost 9
[2 3 6 7 5 0 0 6] cost 4
[3 1 5 7 6 0 4 2] cost 5
[5 5 0 5 2 0 5 3] cost 10
Fitness of the population is measured using n*(n-1)/2 - cost, where n = 8 for a standard chessboard
A sample of the fitness weighting table can be seen below:
[0.05397727 0.04261364 0.05965909 0.05681818 0.05397727 0.05965909
 0.05113636 0.05397727 0.06534091 0.05397727 0.05965909 0.05681818
 0.0625     0.03125    0.05397727 0.06818182 0.06534091 0.05113636]
The parents are selected based on the weights from the fitnesses to make lower cost parents more likely
However, high cost parents are still poss

## Task 2

In task 2, the algorithm will be improved to converge to a solution. The algorithm will also be improved so that convergence is faster. A drawback of the algorithm from task 1 is that it sometimes takes too many generations before convergence and, as such, the algorithm can time out.

The algorithm in task 2 aims to prevent this and improve efficiency.

In [61]:
N = 18
n = 8

# dynamic mutate algorithm adapts using a measure of genetic diversity to either increase or decrease the rate of mutations
# Upper and lower bounds of the mutation rate are also defined above
def dynamic_mutate(pop, pm):        
    for j in range(len(pop)):
        if random.random() <= pm:
            pop[j].state[random.randrange(0,n)] = random.randrange(0,n)
        else:
            pass

# calculates the new mutation probability based on the new population
def calculate_pm(fitness, Pm):
    # parameters used from Vasconcelos et al., 2001
    Vmax = 0.15
    Vmin = 0.005
    Pm_max = 0.25
    Pm_min = 0.001
    
    gdm_max = np.max(fitness)
    gdm_mean = np.mean(fitness)
    gdm = gdm_max/gdm_mean
    
    if gdm > Vmax:
        Pm = 1.1*Pm
    elif gdm < Vmin:
        Pm = Pm/1.1
        
    if Pm > Pm_max:
        Pm = Pm_max
    elif Pm < Pm_min:
        Pm = Pm_min
        
    return Pm

population = initialise_population(N, n)
i = 1
# starting value of Pm at 0.05
pm = 0.05

while contains_solution(population) == False:
    if i % 500 == 0:
        print(str(i) + " Generations completed")
    fit = fitness(population, N)
    select = selection(population, fit)
    newpopulation = []
    for j in select:
        children = crossover(j[0], j[1])
        newpopulation.append(children[0])
        newpopulation.append(children[1])

    population = newpopulation
    pm = calculate_pm(fit, pm)
    dynamic_mutate(population, pm)
    i += 1
    
for state in population:
    if state.is_goal():
        print("Goal State reached after " + str(i) + " Generations")
        print("Population size: " + str(N))
        print("Board size: " + str(n))
        print(state)

500 Generations completed
1000 Generations completed
Goal State reached after 1326 Generations
Population size: 18
Board size: 8
[4, 2, 0, 5, 7, 1, 3, 6] Goal state!


### Comparison between algorithms

The two algorithms are then compared for a board size 7, to speed up the test, and population size 18 over 50 iterations to find the average number of generations taken to find a solution. The hypothesis is that the dynamic mutation algorithm will provide an advantage in converging towards a solution through better placed mutations.
Using 50 iterations minimises the chances of lucky starts/random mutations affecting the outcome of the test

In [67]:
genSGA = 0
N = 18
n = 7

for j in range(50):

    population = initialise_population(N, n)
    i = 1
    while contains_solution(population) == False:
        fit = fitness(population, N)
        select = selection(population, fit)
        newpopulation = []
        for j in select:
            children = crossover(j[0], j[1])
            newpopulation.append(children[0])
            newpopulation.append(children[1])

        population = newpopulation
        mutate(population)
        i += 1

    for state in population:
        if state.is_goal():
            genSGA += i
            
print("Population size: " + str(N) + ", Board size: " + str(n))
print("")
print("Standard Genetic Algorithm")
print("Average no. of generations to complete 50 solutions: " + str(round(genSGA/50, 3)))


genGADM = 0
for j in range(50):

    population = initialise_population(N, n)
    i = 1
    while contains_solution(population) == False:
        fit = fitness(population, N)
        select = selection(population, fit)
        newpopulation = []
        for j in select:
            children = crossover(j[0], j[1])
            newpopulation.append(children[0])
            newpopulation.append(children[1])

        population = newpopulation
        pm = calculate_pm(fit, pm)
        dynamic_mutate(population, pm)
        i += 1

    for state in population:
        if state.is_goal():
            genGADM += i
            
print("Genetic Algorithm with Dynamic Mutation")
print("Average no. of generations to complete 50 solutions: " + str(round(genGADM/50, 3)))

Population size: 18, Board size: 7

Standard Genetic Algorithm
Average no. of generations to complete 50 solutions: 744.06
Genetic Algorithm with Dynamic Mutation
Average no. of generations to complete 50 solutions: 522.94


### Task 3: Explanation of algorithms and implementations

#### Task 1

In task 1, the choice of algorithm was a base level genetic algorithm that was implemented using the outline provided by Russell and Norvig in the course content. The algorithm uses a starting population of 18 as this would optimise both time and space, ensuring that computing power is minimised whilst also ensuring a solution is converged upon quickly.

One of the key implementation choices made for this algorithm was the selection criteria. A simple set of selection criteria are chosen - a probability based selection method which uses the weights found through the fitness function. The fitness function returns the weights for each individual, with a total weight of 1. These weights are used as the probabilities for the parent-pair selection, favouring low-cost solutions whilst also not discounting high-cost solutions. This ensures that diversity is maintained in the population so the solution iterations do not get stuck. Another key decision was the crossover step. In this step, the decision was made for the crossover point to be chosen at random. Once again, this was done to increase the diversity of the solution pool so that a solution could be converged upon faster. Finally, another key decision was the mutation rate. At first, the rate was set to 5%, but after trial-and-error, the rate was found to have more success at 10% as a larger mutation rate would help keep the population heterogeneous and reduce the time taken to find a solution. 

The task 1 algorithm and implementation had some key decisions that makes the algorithm perform well - reaching solutions quickly, whilst minimising the spatial complexity of the solution. However, one issue that the algorithm faces is that the population can homogenise quite quickly, resulting in countless generations in which there is no progresss.

#### Task 2

The goal of task 2 was to use new techniques to improve the existing genetic algorithm. As the solution converging to a local minima had caused issues during my work in task 1, the work of Vasconcelos et al. provided a great starting point from which an adaptive mutation algorithm was used (2001). Conclusions reached by Patil and Bhende mirror this sentiment, as mutation-based algorithms provide a solid starting point to improve the time to convergence and can help maintain genetic diversity in the population (2014). 

Even while keeping the other parameters - board size and population size the same, the dynamic mutation algorithm produced faster results in general. The algorithm followed the psuedocode provided by Vasconcelos et al. to tune the mutation rate according to a genetic diversity parameter (gdm). This parameter is the ratio of the maximum value of the fitness function to the mean of the fitness function. If the maximum value is the mean, then there is no diversity in the population and the parameter is equal to 1. Vasconcelos et al. provided parameters that were used in comparison - based on the gdm, the value of Pm, the probability of mutation, would increase or decrease based on thresholds. That is to say, if genetic diversity is lower, then the probability of mutation increases, and as genetic diversity increases, the parameter correspondingly decreases (2001). The rate of this parameter change may need to be tuned as it may cause excess mutations in states where minimal mutations are needed. 

However, after completing a basic test in the cell above, where the average number of generations to converge to a solution (albeit on a board of n=7 to speed up the overall time required for the test) was obtained over 50 simulations each. Completing 50 simulations reduces the chance of lucky starting seeds or mutations from dominating the results.

There is a substantial improvement by the dynamic mutation algorithm - requiring 29.7% less generations to complete the task. This is in agreement with the findings of Patil and Bhende, where they state that an algorithm similar to the one implemented can converge upon a solution quickly (2014). This reduces the computational requirements of the overall algorithm especially for large search spaces, while maintaining the same level of spatial complexity - only minimal amounts of extra information are stored. Thus, it can be seen that the genetic algorithm with dynamic mutation is a very effective improvement of the algorithm.


### Bibliography

Patil, S. and Bhende, M., 2014. Comparison and analysis of different mutation strategies to improve the performance of genetic algorithm. IJCSIT) International Journal of Computer Science and Information Technologies, 5(3), pp.4669-4673.

Vasconcelos, J. A., Ramirez, J. A., Takahashi, R. H. C. and Saldanha, R. R. Sept. 2001. "Improvements in genetic algorithms," in IEEE Transactions on Magnetics, vol. 37, no. 5, pp. 3414-3417, Sept. 2001, doi: 10.1109/20.952626. 